# CS6301 IOT Project 2

## Fault Diagnostic Tool (Read Side)

This side of the application ingests data from the data generator and ensures it's being imported into the database correctly.

### Before Running this Notebook
1. Ensure the database is running on your localhost. Using docker-compose file in the root of the project directory we can start the InfluxDB with the following command from the root directory:
`dc up timeseriesdb`

2. Turn on the data generator with the following command from the "iot" directory.
`java -jar datagen-2.2-SNAPSHOT.jar -offline`

## Connect to Database

In [1]:
!pip install influxdb

In [27]:
from influxdb import InfluxDBClient
import json

In [28]:
client = InfluxDBClient(host='localhost', port=8086, username='admin', password='password')

In [29]:
# client.drop_database('fault-detector')

In [30]:
client.get_list_database()

[{'name': 'timeseriesdb'}]

## Ingest Data (Example: Toothbrush)
This section simply used as an example for working with InfluxDB.

In [7]:
client.create_database('pyexample')

In [8]:
client.switch_database('pyexample')

In [9]:
json_body = [
    {
        "measurement": "brushEvents",
        "tags": {
            "user": "Carol",
            "brushId": "6c89f539-71c6-490d-a28d-6c5d84c0ee2f"
        },
        "time": "2020-03-28T8:01:00Z",
        "fields": {
            "duration": 127
        }
    },
    {
        "measurement": "brushEvents",
        "tags": {
            "user": "Carol",
            "brushId": "6c89f539-71c6-490d-a28d-6c5d84c0ee2f"
        },
        "time": "2020-03-29T8:04:00Z",
        "fields": {
            "duration": 132
        }
    },
    {
        "measurement": "brushEvents",
        "tags": {
            "user": "Carol",
            "brushId": "6c89f539-71c6-490d-a28d-6c5d84c0ee2f"
        },
        "time": "2020-03-30T8:02:00Z",
        "fields": {
            "duration": 129
        }
    }
]

In [10]:
client.write_points(json_body)

True

In [11]:
results = client.query('SELECT "duration" FROM "pyexample"."autogen"."brushEvents" WHERE time > now() - 4d GROUP BY "user"')

In [12]:
results.keys()

[('brushEvents', {'user': 'Carol'})]

In [13]:
points = results.get_points(tags={'user':'Carol'})

In [14]:
for point in points:
    print("Time: %s, Duration: %i" % (point['time'], point['duration']))

Time: 2020-03-28T08:01:00Z, Duration: 127
Time: 2020-03-29T08:04:00Z, Duration: 132
Time: 2020-03-30T08:02:00Z, Duration: 129


In [15]:
results = client.query('SELECT "duration" FROM "pyexample"."autogen"."brushEvents" WHERE time > now() - 4d GROUP BY "brushId"')
points = results.get_points(tags={'brushId': '6c89f539-71c6-490d-a28d-6c5d84c0ee2f'})
brush_usage_total = 0
for point in points:
    brush_usage_total = brush_usage_total + point['duration']

if brush_usage_total > 350:
    print("You've used your brush head for %s seconds, more than the recommended amount! Time to replace your brush head!" % brush_usage_total)

You've used your brush head for 388 seconds, more than the recommended amount! Time to replace your brush head!


## Ingest Data (Example: Offline Data)
In this section we will ingest the data from a text file for exploration purposes. Normally this data would be ingesteddirctly from the Java data generator.

In [31]:
client.drop_database('timeseriesdb')
client.create_database('timeseriesdb')
client.switch_database('timeseriesdb')

In [10]:
f = open('data/offline-train.txt', 'r')
lines = f.readlines()
f.close()

In [11]:
len(lines)

26452

In [12]:
measurement = 'gear_metrics'

In [13]:
data = []

In [14]:
x = json.loads(lines[0])
x

{'metric': 'offline',
 'timestamp': 0,
 'label': 0,
 'sr': 97656.0,
 'rate': 25.0,
 'gs': 0.8407559,
 'load': '270.0'}

In [15]:
data = []
for line in lines:
    json_line = json.loads(line)
    # Form: 'gear_metrics,metric=offline label=0,sr=97656.0,rate=25.0,gs=0.8407559,load=270.0,timestamp=0'
    data.append("{},metric={} label={},sr={},rate={},gs={},load={},timestamp={}".format(measurement, json_line['metric'], json_line['label'], json_line['sr'], json_line['rate'], json_line['gs'], json_line['load'], json_line['timestamp']))
    
    

In [16]:
data[:10]

['gear_metrics,metric=offline label=0,sr=97656.0,rate=25.0,gs=0.8407559,load=270.0,timestamp=0',
 'gear_metrics,metric=offline label=0,sr=97656.0,rate=25.0,gs=0.5152432,load=270.0,timestamp=1',
 'gear_metrics,metric=offline label=0,sr=97656.0,rate=25.0,gs=-0.03834483,load=270.0,timestamp=2',
 'gear_metrics,metric=offline label=0,sr=97656.0,rate=25.0,gs=1.184862,load=270.0,timestamp=3',
 'gear_metrics,metric=offline label=0,sr=97656.0,rate=25.0,gs=0.8497145,load=270.0,timestamp=4',
 'gear_metrics,metric=offline label=0,sr=97656.0,rate=25.0,gs=-0.3333637,load=270.0,timestamp=5',
 'gear_metrics,metric=offline label=0,sr=97656.0,rate=25.0,gs=0.0906198,load=270.0,timestamp=6',
 'gear_metrics,metric=offline label=0,sr=97656.0,rate=25.0,gs=-0.3867708,load=270.0,timestamp=7',
 'gear_metrics,metric=offline label=0,sr=97656.0,rate=25.0,gs=-0.4381106,load=270.0,timestamp=8',
 'gear_metrics,metric=offline label=0,sr=97656.0,rate=25.0,gs=-0.7925295,load=270.0,timestamp=9']

In [17]:
client.write_points(data, database='timeseriesdb', time_precision='ms', batch_size=100, protocol='line')

True

In [25]:
results = client.query('SELECT label, sr, rate, gs, load, timestamp FROM timeseriesdb.autogen.gear_metrics')

In [26]:
len(results.raw['series'][0]['values'])

234

In [20]:
results.raw['series'][0]['values'][0:10]

[['2020-03-22T22:08:28.898Z', 0, 97656, 25, -0.6083242, 270, 99],
 ['2020-03-22T22:08:28.953Z', 0, 97656, 25, -1.005764, 270, 199],
 ['2020-03-22T22:08:28.969Z', 0, 97656, 25, -1.57044, 270, 299],
 ['2020-03-22T22:08:28.981Z', 0, 97656, 25, 0.3499642, 270, 399],
 ['2020-03-22T22:08:28.991Z', 0, 97656, 25, 0.1315735, 270, 499],
 ['2020-03-22T22:08:29.002Z', 0, 97656, 25, -0.7119758, 270, 599],
 ['2020-03-22T22:08:29.011Z', 0, 97656, 25, -0.4874076, 270, 699],
 ['2020-03-22T22:08:29.02Z', 0, 97656, 25, -1.071809, 270, 799],
 ['2020-03-22T22:08:29.03Z', 0, 97656, 25, -0.4716315, 270, 899],
 ['2020-03-22T22:08:29.041Z', 0, 97656, 25, -0.09621894, 270, 999]]

In [37]:
for point in points:
    print("Time: {}, gs: {}".format(point['timestamp'], point['gs']))

## Visualize Ingested Data
In this section we visualize the data ingested from the Java data generator. 